In [1]:
import subprocess

In [3]:
print(subprocess.Popen("echo Hello World", shell=True, stdout=subprocess.PIPE).stdout.read())

b'Hello World\r\n'


In [24]:
jdk_path = subprocess.Popen("echo %JAVA_HOME%", shell=True, stdout=subprocess.PIPE).stdout.readline()    # to make sure jdk path is set
print(jdk_path)

In [26]:
weka_path="weka-3-4\weka.jar"                    # don't change these unless you changed the location of these folders

In [28]:
COMMONS_CLI="lib\commons-cli-1.0.jar"            # don't change these unless you changed the location of these folders

In [29]:
JMRC="lib\jmrc.jar"                              # don't change these unless you changed the location of these folders

In [116]:
appDir = "E:\CiperLabs\\fiver\scripts\personality recognizer\PersonalityRecognizer"     #Directory where the java application is located

In [126]:
LIBS=weka_path+";"+COMMONS_CLI+";"+JMRC+";"+'bin\\'     #+appDir+";"

In [128]:
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [200]:
#Change the properties here to change the model parameters and files


# -d,--directory		Corpus analysis mode. Input must be a directory with 
#                          multiple text files, features are standardized over 
#                          the corpus and the recognizer outputs a personality 
#                          estimate for each text file.

input_type_is_dir = "-d"  #("set "-d" if folder or "" if file")

# -i,--input       	Input file or directory (required)
input_file_folder = "examples" 

#-m,--model       	Model to use for computing scores (default 4). Options:
#  	              				1 = Linear Regression
#                				2 = M5' Model Tree
#              					3 = M5' Regression Tree
#              					4 = Support Vector Machine with Linear Kernel (SMOreg)

model = "3" 

# -t,--type		Selects the type of model to use (default 1). The appropriate
#                         		model depends on the language sample (written or 
#   					spoken), and whether observed personality (as perceived 
#   					by external judges) or self-assessed personality (the 
#   					writer/speaker's perception) needs to be estimated from the 
#   					text. Options:
#   						1 = Observed personality from spoken language
#                                 		2 = Self-assessed personality from written language
model_type = "2"

In [201]:
def execPR():
    output = []
    # enter the app directory
    with cd(appDir):
       # we are in app directory when exiting the block we exit from the directory
        command = 'java -Xmx512m -classpath '+ LIBS+ ' recognizer.PersonalityRecognizer '+ input_type_is_dir + ' -i '+input_file_folder + ' -m '+model+' -t '+model_type
        p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        print(command)
        for line in p.stdout.readlines():
            output.append(line.decode("utf-8"))
        retval = p.wait()
    return output

In [202]:
outputstream = execPR()

java -Xmx512m -classpath weka-3-4\weka.jar;lib\commons-cli-1.0.jar;lib\jmrc.jar;bin\ recognizer.PersonalityRecognizer -d -i examples -m 3 -t 2


In [206]:
print(outputstream)

['LIWC dictionary loaded (68 lexical categories)\r\n', 'Loading MRC Psycholinguistic Database...\r\n', '120038 words loaded.\r\n', 'Loading model E:\\CiperLabs\\fiver\\scripts\\personality recognizer\\PersonalityRecognizer\\lib\\models\\self\\M5P-R\\std-extra.model...\r\n', 'Loading model E:\\CiperLabs\\fiver\\scripts\\personality recognizer\\PersonalityRecognizer\\lib\\models\\self\\M5P-R\\std-ems.model...\r\n', 'Loading model E:\\CiperLabs\\fiver\\scripts\\personality recognizer\\PersonalityRecognizer\\lib\\models\\self\\M5P-R\\std-agree.model...\r\n', 'Loading model E:\\CiperLabs\\fiver\\scripts\\personality recognizer\\PersonalityRecognizer\\lib\\models\\self\\M5P-R\\std-consc.model...\r\n', 'Loading model E:\\CiperLabs\\fiver\\scripts\\personality recognizer\\PersonalityRecognizer\\lib\\models\\self\\M5P-R\\std-open.model...\r\n', 'Reading directory E:\\CiperLabs\\fiver\\scripts\\personality recognizer\\PersonalityRecognizer\\examples...\r\n', '\r\n', 'Computing features for file 

In [204]:
output=[]                 #{filename,textprop,LIWC,MRC,extra,emoti,agree,consc,openn}
results = []
outStarted = False
for line in outputstream:
    if(line.startswith("Computing features for file")):
        name = line[28:][:-5]
        out = {}
        out["name"] =name
        index_of_arr = outputstream.index(line)
        out["textprop"] = outputstream[index_of_arr+1][:-2]
        out["LIWC"] =outputstream[index_of_arr+2][:-2]
        out["MRC"] = outputstream[index_of_arr+3][:-2]
        output.append(out)
        print(name)
    elif(line.startswith("File              \tExtra\tEmoti\tAgree\tConsc\tOpenn\r\n")):
        outStarted = True
    elif(line.startswith("\r\n")):
        outStarted = False
    elif(outStarted):
        results.append(line[:-2])

Amy.txt
John.txt
Matt.txt
Rob.txt


In [205]:
for i in range(len(output)):
    res = results[i].split(	)                # Splitting by tab
    output[i]["extra"] = res[1]
    output[i]["emoti"] = res[2]
    output[i]["agree"] = res[3]
    output[i]["consc"] = res[4]
    output[i]["openn"] = res[5]

In [199]:
output

[{'name': 'Amy.txt',
  'textprop': 'Input text splitted into 551 words and 36 sentences',
  'LIWC': 'LIWC features computed: 70',
  'MRC': 'MRC features computed: 14',
  'extra': '4.64',
  'emoti': '3.682',
  'agree': '4.697',
  'consc': '4.614',
  'openn': '4.746'},
 {'name': 'John.txt',
  'textprop': 'Input text splitted into 808 words and 67 sentences',
  'LIWC': 'LIWC features computed: 70',
  'MRC': 'MRC features computed: 14',
  'extra': '4.865',
  'emoti': '4.219',
  'agree': '4.747',
  'consc': '4.633',
  'openn': '4.694'},
 {'name': 'Matt.txt',
  'textprop': 'Input text splitted into 322 words and 11 sentences',
  'LIWC': 'LIWC features computed: 70',
  'MRC': 'MRC features computed: 14',
  'extra': '4.764',
  'emoti': '3.24',
  'agree': '4.571',
  'consc': '4.788',
  'openn': '5.095'},
 {'name': 'Rob.txt',
  'textprop': 'Input text splitted into 467 words and 22 sentences',
  'LIWC': 'LIWC features computed: 70',
  'MRC': 'MRC features computed: 14',
  'extra': '4.858',
  'em